In [40]:
import os 
import glob 

root_dir = '/checkpoint/apratik/data/data/apartment_0/default/no_noise/mul_traj_200'
combine = 2
new_dir = f'/checkpoint/apratik/data/data/apartment_0/default/no_noise/mul_traj_200_combined_{combine}'

if not os.path.isdir(new_dir):
    os.makedirs(new_dir)
    
# check valid trajectories
def basic_sanity(traj_path):
    def ll(x, y, ext):
        return len(glob.glob(os.path.join(x, y, f'*{ext}')))
    # TODO: add base_pose sanity check here
    return ll(traj_path, 'rgb', '.jpg') == ll(traj_path, 'seg', '.npy') == ll(traj_path, 'depth', '.npy')

valid_traj = []
for x in range(201):
    traj_path = os.path.join(root_dir, str(x))
    if os.path.isdir(traj_path) and basic_sanity(traj_path):
        valid_traj.append(traj_path)

In [ ]:
from shutil import copyfile
import json

def acopyjson(a, b, c):
    with open(a, "r") as f:
        dat = json.load(f)
        
    with open(b, "r") as f:
        dat2 = json.load(f)
        
    ctr = len(dat.keys())
#     print(f'before {len(dat.keys()), len(dat2.keys())}')
    for k, v in dat2.items():
        dat[ctr] = v
        ctr += 1
    
#     print(f'after {len(dat.keys())}')
    with open(c, "w") as fp:
        json.dump(dat, fp)

def acopydir(a, b, c, ext):
    if not os.path.isdir(c):
        os.makedirs(c)
    
    fsa = [x for x in glob.glob(os.path.join(a, f'*{ext}'))]
    
    ctr = 0
    for f in fsa:
        fname = f.split('/')[-1]
        copyfile(f, os.path.join(c, fname))
        ctr += 1

    fsb = [x for x in glob.glob(os.path.join(b, f'*{ext}'))] 
    for f in fsb:
        fname = "{:05d}{}".format(ctr, ext)
        ctr += 1
        copyfile(f, os.path.join(c, fname))
    
#     print(f'acopydir {len(fsa), len(fsb)}')
    
    
def combine(traj_a, traj_b, traj_ab):
    if not os.path.isdir(traj_ab):
        os.makedirs(traj_ab)
    
    # copy over rgb, depth, seg from traj a
    # copy over rgb, d, seg from traj b but with new names 
    acopydir(os.path.join(traj_a, 'rgb'), os.path.join(traj_b, 'rgb'), os.path.join(traj_ab, 'rgb'), '.jpg')   
    acopydir(os.path.join(traj_a, 'depth'), os.path.join(traj_b, 'depth'), os.path.join(traj_ab, 'depth'), '.npy')
    acopydir(os.path.join(traj_a, 'seg'), os.path.join(traj_b, 'seg'), os.path.join(traj_ab, 'seg'), '.npy')
    acopyjson(os.path.join(traj_a, 'data.json'), os.path.join(traj_b, 'data.json'), os.path.join(traj_ab, 'data.json'))
    
ctr = 1
for x in range(0, len(valid_traj), 2):
    a = valid_traj[x]
    b = valid_traj[x+1]
    if os.path.isdir(a) and os.path.isdir(b):
        print(f'combining {a} and {b}')
        combine(a, b, os.path.join(new_dir, str(ctr)))
        ctr += 1
    

combining /checkpoint/apratik/data/data/apartment_0/default/no_noise/mul_traj_200/2 and /checkpoint/apratik/data/data/apartment_0/default/no_noise/mul_traj_200/3
